<a href="https://colab.research.google.com/github/Chelovek2002/BA_Project_Team_E/blob/main/Project_Team_E_'Easy'.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Мы - банк, который доставляет карточки потребителям внутри одного города. Город поделён на кварталы.

Каждый день в квартале может возникнуть заказ на доставку.

Банк в начале недели выбирает покупает n смен работы курьеров (каждая смена длится до 12 часов). Задача курьера - доставить карточки как максимальному числу клиентов.

Банк платит курьерам n рублей за каждый проеханый километр

In [ ]:
!pip install h3
!pip install h3pandas
!pip install boto3
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 75 kB 2.2 MB/s 
     |████████████████████████████████| 1.0 MB 9.4 MB/s 
     |████████████████████████████████| 6.3 MB 31.0 MB/s 
     |████████████████████████████████| 16.7 MB 601 kB/s 
  Created wheel for h3pandas: filename=h3pandas-0.2.3-py3-none-any.whl size=17136 sha256=36057e770a525f7beef6361275387ed7b436433e8134f1058834a08127bfd34a
  Stored in directory: /root/.cache/pip/wheels/59/d8/b5/d5406a801c743e39323e3305611d8caf558077b6caf7e8858f
Successfully built h3pandas
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 4.2 MB/s 
     |████████████████████████████████| 79 kB 273 kB/s 
     |█████████

In [ ]:
pip install folium matplotlib mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 6.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.31.6 requires protobuf<4.0.0dev,>=3.12.0; python_version > "3", but you have protobuf 4.21.9 which is incompatible.


In [ ]:
import h3
import h3pandas 
import boto3
import pandas as pd
import geopandas as gpd
import numpy as np

# Получаем данные 
У нас будет использоваться 2 датасета, первый - гексагоны, куда попадают заявки на продукты с вероятностью попадания заявки. Второй - матрица расстояний между гексагонами 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_request = pd.read_csv('/content/drive/MyDrive/requests_df.csv')
dist_matrix = pd.read_csv('/content/drive/MyDrive/dist_matr.csv').set_index('index')

In [ ]:
df_request.head(5)

,hex,probs
0,882175aa1bfffff,0.172043
1,882175aacdfffff,0.069892
2,882175aa13fffff,0.069892
3,882175a167fffff,0.057348
4,882175a123fffff,0.051971


In [ ]:
dist_matrix.head(5)

,882175aa1bfffff,882175aacdfffff,882175aa13fffff,882175a167fffff,882175a123fffff,882175aa17fffff,882175aa11fffff,882175a12dfffff,882175a135fffff,882175aac5fffff,...,882175b829fffff,882175b6adfffff,882175a647fffff,882175a23dfffff,8821751817fffff,8821751413fffff,8821753159fffff,8821751a1dfffff,882175aa81fffff,882175a815fffff
index,,,,,,,,,,,,,,,,,,,,,
882175aa1bfffff,0.0,743.0,779.0,5712.0,2988.0,1751.0,1124.0,4659.0,3250.0,1695.0,...,15492.0,45171.0,31200.0,25323.0,28425.0,20670.0,25671.0,28425.0,5493.0,9520.0
882175aacdfffff,743.0,0.0,887.0,5416.0,2693.0,1928.0,1572.0,4364.0,2955.0,1170.0,...,15953.0,44437.0,30905.0,25027.0,28886.0,20899.0,25184.0,28886.0,4874.0,9032.0
882175aa13fffff,779.0,887.0,0.0,6249.0,3525.0,1441.0,1137.0,5196.0,3787.0,1396.0,...,16076.0,45324.0,31737.0,25860.0,29009.0,20391.0,24951.0,29009.0,5236.0,8800.0
882175a167fffff,5712.0,5416.0,6249.0,0.0,3553.0,7200.0,6836.0,1243.0,4282.0,6586.0,...,20528.0,45477.0,31948.0,26071.0,30315.0,26141.0,30600.0,30315.0,10290.0,14448.0
882175a123fffff,2988.0,2693.0,3525.0,3553.0,0.0,4476.0,4112.0,2655.0,1474.0,3863.0,...,17811.0,43422.0,29893.0,24016.0,28260.0,23396.0,27877.0,28260.0,7567.0,11725.0


Давайте посмотрим на наши заявки

In [ ]:
df_request.set_index('hex').h3.h3_to_geo_boundary().explore('probs')

In [ ]:
time_windows = [[9, 11], [11,13], [13,15], [15,17], [17, 19], [19,21]]
probs = [0.2, 0.1, 0.05, 0.1, 0.25, 0.3]


def get_meet_sample(req_df, n):
  np.random.choice(req_df.hex, p = req_df.probs)



SyntaxError: ignored